In [1]:
!pip install aiogram aiosqlite apscheduler nest_asyncio

In [16]:
!pip install gigachat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [19]:
from google.colab import userdata
token = userdata.get('TelegramBot')
GigaChatKey = userdata.get('GigaChatKey')

In [17]:
from gigachat import GigaChat

In [4]:
from aiogram import Bot, Dispatcher
from aiogram.client.default import DefaultBotProperties
from aiogram.enums import ParseMode
from aiogram.fsm.storage.memory import MemoryStorage
from aiogram import Router, F
from aiogram.filters import CommandStart, Command, CommandObject
from aiogram.types import Message
from aiogram.types import KeyboardButton, ReplyKeyboardMarkup, KeyboardButtonPollType
from aiogram.utils.keyboard import ReplyKeyboardBuilder
from aiogram.types import InlineKeyboardMarkup, InlineKeyboardButton, WebAppInfo
from aiogram.utils.keyboard import InlineKeyboardBuilder
from aiogram.types import BotCommand, BotCommandScopeDefault
from aiogram.types import CallbackQuery
from aiogram.utils.chat_action import ChatActionSender
from aiogram.types import ReplyKeyboardRemove
from aiogram.fsm.context import FSMContext
from aiogram.fsm.state import State, StatesGroup
from typing import Any, Awaitable, Callable, Dict
from aiogram import BaseMiddleware
from aiogram.types import TelegramObject
from html import escape
import asyncio
import logging
import aiosqlite
from apscheduler.schedulers.asyncio import AsyncIOScheduler

In [26]:
#Определим состояния:
class Form(StatesGroup):
    choice = State()
    books = State()

#Включаем логирование
logging.basicConfig(force=True, level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

bot = Bot(token=token, default=DefaultBotProperties(parse_mode=ParseMode.HTML))
dp = Dispatcher(storage=MemoryStorage())

listuser = {}

class SomeMiddleware(BaseMiddleware):
    async def __call__(
        self,
        handler: Callable[[TelegramObject, Dict[str, Any]], Awaitable[Any]],
        event: TelegramObject,
        data: Dict[str, Any]
    ) -> Any:
        if data['event_update'].message.text not in ('/start', '/info'):
            id = data['event_update'].message.chat.id
            async with aiosqlite.connect('data.db') as db:
                async with db.execute("SELECT id FROM users WHERE id = ?", (id,)) as cursor:
                    if await cursor.fetchone() is None:
                        await bot.send_message(chat_id=id, text='Вы не зарегистрированы! Зарегистрируйтесь, используя команду /start.')
                        return
        result = await handler(event, data)
        return result

async def start_db():
    # Подключение к базе данных (если файл не существует, он будет создан)
    async with aiosqlite.connect('data.db') as db:
        # Создание таблицы, если она не существует
        await db.execute('''
            CREATE TABLE IF NOT EXISTS users (
                id INTEGER NOT NULL PRIMARY KEY,
                statusrem BOOLEAN,
                name CHAR(100)
            )
        ''')
        await db.commit() # Сохранение изменений
        await db.execute('''
            CREATE TABLE IF NOT EXISTS books (
                id INTEGER NOT NULL PRIMARY KEY,
                title CHAR(100),
                url char(300)
            )
        ''')
        await db.commit() # Сохранение изменений
        i = 0
        books = {'https://drive.google.com/file/d/13a_WAY4eb5PyF-ovr6SaC_4p0i1BKPRW/view?usp=sharing': 'Харшман — Развитие нестандартного мышления',
                 'https://drive.google.com/file/d/1BIak4wDMX2oMV15Ey6DBj4MlM2ZHx7X7/view?usp=sharing': 'Уемов — Логические ошибки',
                 'https://docs.google.com/document/d/1SWjD6QnHcyBZE9jRSf618MN3VQ_WzSsR/edit?usp=sharing&ouid=102921155101912639531&rtpof=true&sd=true':
                 'Вуджек — Тренировка ума',
                 'https://drive.google.com/file/d/1UuhdNRTgpJbZTNXvBmgIYygtR1o33hba/view?usp=sharing':'Ивин А. А. — Логика',
                 'https://drive.google.com/file/d/1r5M5i_gET4EnHR80UPa2lpOEa6AayDQu/view?usp=sharing':'Слоун Пол, Дез Макхейл — Задачи на нестандартное мышление'}
        for url in books.keys():
            await db.execute('INSERT INTO books (id, title, url) VALUES (?, ?, ?) ON CONFLICT DO NOTHING', (i, books[url], url))
            await db.commit()
            i += 1
        await db.execute('''
            CREATE TABLE IF NOT EXISTS books_to_hide (
                id_user INTEGER,
                id_book INTEGER,
                PRIMARY KEY (id_user, id_book)
            )
        ''')
        await db.commit() # Сохранение изменений

@dp.message(CommandStart())
async def cmd_start(message: Message, state: FSMContext):
    async with aiosqlite.connect('data.db') as db:
        async with db.execute("SELECT id, name FROM users WHERE id = ?", (message.from_user.id,)) as cursor:
            row = await cursor.fetchone()
            if row is None:
                await message.answer(f'Здравствуйте, {message.from_user.last_name} {message.from_user.first_name}! Вы зарегистрированы')
                await message.answer(text='Этот бот задумывался как помощник в развитии логического мышления с применением ИИ и справочной информацией.\nРеализован был только справочный функционал, в меру сил.\nБЯМ в лице гигачата задействована для ответов на незапланированные сообщения.\nМашина состояний есть, но я не до конца поняла, как правильно её реализовать')
                await cursor.execute('INSERT INTO users (id, statusrem, name) VALUES (?, ?, ?)',
                 (message.from_user.id, False, message.from_user.last_name + ' ' + message.from_user.first_name))
                await db.commit()
            else:
                name = row[1]
                await message.answer(f'Привет {name}! Вы уже зарегистрированы!')
    await state.set_state(Form.choice)
    builder = InlineKeyboardBuilder()
    builder.button(text='Книги', callback_data='books')
    builder.button(text='Сайты', callback_data='sites')
    await message.answer('Выберите действие:', reply_markup=builder.as_markup())

@dp.callback_query(F.data == 'books', Form.choice)
async def books_lst(call: CallbackQuery, state: FSMContext):
    builder = InlineKeyboardBuilder()
    builder.button(text='Оставшиеся', callback_data='remain')
    builder.button(text='Скрыто', callback_data='hidden')
    builder.adjust(1, 1)
    await bot.send_message(chat_id=call.from_user.id, text='Вот ваши книги:', reply_markup=builder.as_markup())
    await state.set_state(Form.books)

@dp.callback_query(F.data == 'sites', Form.choice)
async def sites_lst(call: CallbackQuery, state: FSMContext):
    builder = InlineKeyboardBuilder()
    builder.button(text='Серые клеточки', url='https://www.graycell.ru/index.html')
    builder.button(text="Ребусы ВИКВИЗ", url='https://viquiz.ru/theme/visual/rebus/')
    builder.button(text='Абсолютно бесполезный сайт', url='https://absite.ru/')
    builder.adjust(2, 1)
    await bot.send_message(chat_id=call.from_user.id, text='Выберите сайт:', reply_markup=builder.as_markup())

@dp.callback_query(F.data == 'remain', Form.books)
async def books_lst(call: CallbackQuery, state: FSMContext):
    async with aiosqlite.connect('data.db') as db:
        async with db.execute("SELECT id, title, url FROM books WHERE NOT EXISTS (SELECT id_user FROM books_to_hide WHERE id_book = books.id AND id_user = ?)",
                              (call.from_user.id,)) as cursor:
            table = await cursor.fetchall()
            if len(table) == 0:
                msg = await bot.send_message(chat_id=call.from_user.id, text='Вы всё прочитали)')
            else:
                builder = InlineKeyboardBuilder()
                for row in table:
                    builder.button(text=row[1], url=row[2])
                builder.adjust(1, 1, 1, 1, 1, 1)
                msg = await bot.send_message(chat_id=call.from_user.id, text='Список непрочитанных книг:', reply_markup=builder.as_markup())
            await asyncio.sleep(30)
            await msg.delete()
    await state.clear()

@dp.callback_query(F.data == 'hidden', Form.books)
async def books_lst(call: CallbackQuery, state: FSMContext):
    async with aiosqlite.connect('data.db') as db:
        async with db.execute("SELECT id, title, url FROM books WHERE EXISTS (SELECT id_user FROM books_to_hide WHERE id_book = books.id AND id_user = ?)",
                              (call.from_user.id,)) as cursor:
            table = await cursor.fetchall()
            if len(table) == 0:
                msg = await bot.send_message(chat_id=call.from_user.id, text='Тут ничего нет')
            else:
                builder = InlineKeyboardBuilder()
                for row in table:
                    builder.button(text=row[1], url=row[2])
                builder.adjust(1, 1, 1, 1, 1)
                msg = await bot.send_message(chat_id=call.from_user.id, text='Список скрытых книг:', reply_markup=builder.as_markup())
            await asyncio.sleep(30)
            await msg.delete()
    await state.clear()

@dp.message(Command('info'))
async def cmd_info(message: Message, state: FSMContext):
    await bot.send_message(chat_id=message.from_user.id,
    text='Этот бот задумывался как помощник в развитии логического мышления с применением ИИ и справочной информацией.\nРеализован был только справочный функционал, в меру сил. Например, списков книг два, но перенос из одного в другой не реализован.\nБЯМ в лице гигачата задействована для ответов на незапланированные сообщения.\nМашина состояний есть, но я не до конца поняла, как правильно её реализовать')

async def start_bot():
    commands = [BotCommand(command='start', description='Начало работы'),
                BotCommand(command='info', description='О боте'),]
    await bot.set_my_commands(commands, BotCommandScopeDefault())

@dp.message()
async def prtext(message: Message, state: FSMContext):
    with GigaChat(credentials=GigaChatKey, verify_ssl_certs=False) as giga:
        response = giga.chat('Ты — чат-бот для развития логического мышления пользователей. Ответь на сообщение одного из них: '+message.text)
        await message.answer(response.choices[0].message.content)

async def main(): #Основная асинхронная функция, которая будет запускаться при старте бота.
    #scheduler = AsyncIOScheduler(timezone='Europe/Moscow')
    #job = scheduler.add_job(send_msg, 'interval', seconds=10, args=(dp,))
    #scheduler.start()
    dp.message.outer_middleware(SomeMiddleware())
    dp.startup.register(start_db)
    dp.startup.register(start_bot)
    try:
      print("Бот запущен...")
      await bot.delete_webhook(drop_pending_updates=True)
      await dp.start_polling(bot, allowed_updates=dp.resolve_used_update_types()) #запускаем бота в режиме опроса (polling). Бот начинает непрерывно запрашивать обновления с сервера Telegram и обрабатывать их
    finally:
      #scheduler.remove_job(job.id)
      await bot.session.close()
      print("Бот остановлен")

asyncio.run(main())

Бот запущен...


2025-01-22 18:43:58,729 - aiogram.dispatcher - INFO - Start polling
2025-01-22 18:43:58,831 - aiogram.dispatcher - INFO - Run polling for bot @eh_he_bot id=5568556426 - 'test_bot'
2025-01-22 18:44:02,462 - aiogram.event - INFO - Update id=130927598 is handled. Duration 751 ms by bot id=5568556426
2025-01-22 18:44:09,890 - aiogram.dispatcher - WARNING - Received SIGINT signal
2025-01-22 18:44:09,894 - aiogram.dispatcher - INFO - Polling stopped for bot @eh_he_bot id=5568556426 - 'test_bot'
2025-01-22 18:44:09,896 - aiogram.dispatcher - INFO - Polling stopped


Бот остановлен
